# Music recommender system

One of the most used machine learning algorithms is recommendation systems. A **recommender** (or recommendation) **system** (or engine) is a filtering system which aim is to predict a rating or preference a user would give to an item, eg. a film, a product, a song, etc.

Which type of recommender can we have?   

There are two main types of recommender systems: 
- Content-based filters
- Collaborative filters
  
> Content-based filters predicts what a user likes based on what that particular user has liked in the past. On the other hand, collaborative-based filters predict what a user like based on what other users, that are similar to that particular user, have liked.

### 1) Content-based filters

Recommendations done using content-based recommenders can be seen as a user-specific classification problem. This classifier learns the user's likes and dislikes from the features of the song.

The most straightforward approach is **keyword matching**.

In a few words, the idea behind is to extract meaningful keywords present in a song description a user likes, search for the keywords in other song descriptions to estimate similarities among them, and based on that, recommend those songs to the user.

*How is this performed?*

In our case, because we are working with text and words, **Term Frequency-Inverse Document Frequency (TF-IDF)** can be used for this matching process.
  
We'll go through the steps for generating a **content-based** music recommender system.

### Importing required libraries

First, we'll import all the required libraries.

In [485]:
import numpy as np
import pandas as pd

In [486]:
from typing import List, Dict

We have already used the **TF-IDF score before** when performing Twitter sentiment analysis. 

Likewise, we are going to use TfidfVectorizer from the Scikit-learn package again.

In [487]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import distance

### Dataset

So imagine that we have the [following dataset](https://www.kaggle.com/mousehead/songlyrics/data#). 

This dataset contains name, artist, and lyrics for *57650 songs in English*. The data has been acquired from LyricsFreak through scraping.

In [488]:
songs = pd.read_csv('songdata.csv', nrows=1000)

In [489]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


Because of the dataset being so big, we are going to resample only 5000 random songs.

In [490]:
# songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)
songs = songs.drop('link', axis=1)

We can notice also the presence of `\n` in the text, so we are going to remove it.

In [491]:
songs['text'] = songs['text'].str.replace(r'\n', '', regex=True)

In [492]:
songs['text']

0      Look at her face, it's a wonderful face  And i...
1      Take it easy with me, please  Touch me gently ...
2      I'll never know why I had to go  Why I had to ...
3      Making somebody happy is a question of give an...
4      Making somebody happy is a question of give an...
                             ...                        
995    Here I am again  Waiting on the moment you dec...
996    Oh  Oh  Oh oh oh yeah  Oh no  Yeah    Shorty a...
997    [Chorus]  You're my shining star  That is what...
998    Ignorance of people purchasing diamonds and ne...
999    When you come back I won't be here  She said a...
Name: text, Length: 1000, dtype: object

After that, we use TF-IDF vectorizerthat calculates the TF-IDF score for each song lyric, word-by-word. 

Here, we pay particular attention to the arguments we can specify.

In [493]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [494]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

*How do we use this matrix for a recommendation?* 

We now need to calculate the similarity of one lyric to another. We are going to use **cosine similarity**.

We want to calculate the cosine similarity of each item with every other item in the dataset. So we just pass the lyrics_matrix as argument.

In [495]:
# print(lyrics_matrix[0])

In [496]:
cosine_similarities = cosine_similarity(lyrics_matrix) 
print(cosine_similarities)
print(cosine_similarities.shape)


[[1.         0.00199367 0.00837601 ... 0.01018171 0.05254204 0.01583982]
 [0.00199367 1.         0.00476219 ... 0.03191786 0.01568829 0.02211107]
 [0.00837601 0.00476219 1.         ... 0.02310628 0.00784733 0.01775392]
 ...
 [0.01018171 0.03191786 0.02310628 ... 1.         0.02310019 0.15968013]
 [0.05254204 0.01568829 0.00784733 ... 0.02310019 1.         0.01009026]
 [0.01583982 0.02211107 0.01775392 ... 0.15968013 0.01009026 1.        ]]
(1000, 1000)


In [497]:
# distance_matrix = distance.cdist(lyrics_matrix, lyrics_matrix, metric='euclidean')
# print(distance_matrix)

Once we get the similarities, we'll store in a dictionary the names of the 50  most similar songs for each song in our dataset.

In [498]:
similarities = {}

In [499]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

# print(similarities)

After that, all the magic happens. We can use that similarity scores to access the most similar items and give a recommendation.

For that, we'll define our Content based recommender class.

In [500]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended (Cosine) songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

Now, instantiate class

In [501]:
recommedations = ContentBasedRecommender(similarities)

Then, we are ready to pick a song from the dataset and make a recommendation.

In [502]:
recommendation = {
    "song": songs['song'].iloc[1],
    "number_songs": 8
}

In [503]:
recommedations.recommend(recommendation)

The 8 recommended (Cosine) songs for Andante, Andante are:
Number 1:
Gonna Sing You My Lovesong by ABBA with 0.176 similarity score
--------------------
Number 2:
Cowboy Song by Arlo Guthrie with 0.13 similarity score
--------------------
Number 3:
Making Love Out Of Nothing At All by Air Supply with 0.124 similarity score
--------------------
Number 4:
Dance by ABBA with 0.119 similarity score
--------------------
Number 5:
Don't Let It Get You Down by America with 0.117 similarity score
--------------------
Number 6:
Strong Strong Wind by Air Supply with 0.115 similarity score
--------------------
Number 7:
Power Of Love by Arlo Guthrie with 0.115 similarity score
--------------------
Number 8:
Make It Right by Air Supply with 0.113 similarity score
--------------------


And we can pick another random song and recommend again:

In [520]:
recommendation2 = {
    "song": songs['song'].iloc[800],
    "number_songs": 8
}

In [521]:
recommedations.recommend(recommendation2)

The 8 recommended (Cosine) songs for I Ride On Old Pain are:
Number 1:
Same Old Song And Dance by Aerosmith with 0.174 similarity score
--------------------
Number 2:
Old Friend by Allman Brothers Band with 0.154 similarity score
--------------------
Number 3:
Milkcow Blues by Aerosmith with 0.142 similarity score
--------------------
Number 4:
Let Me Ride by Allman Brothers Band with 0.132 similarity score
--------------------
Number 5:
The Motorcycle Song by Arlo Guthrie with 0.128 similarity score
--------------------
Number 6:
When The Cactus Is In Bloom by Arlo Guthrie with 0.124 similarity score
--------------------
Number 7:
Draw The Line by Aerosmith with 0.123 similarity score
--------------------
Number 8:
Down To The Water by America with 0.122 similarity score
--------------------


In [506]:
# corpus = [
#      'This is the first document is asd  ssa.',
#      'This is the first document is.'
#  ]

# vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')

In [507]:
# X = vectorizer.fit_transform(corpus)
# print(X)
# print(vectorizer.vocabulary_) # dictionary of words and their index

In [508]:
# print(vectorizer.get_stop_words())
lyrics_matrix.shape

(1000, 8734)

In [509]:
# cosine_similarities = cosine_similarity(X) 
# print(cosine_similarities)

In [510]:
array = []
# print(X.shape)
for i in range(lyrics_matrix.shape[0]):
   sub_array = []
   for j in range(lyrics_matrix.shape[1]):
      sub_array.append(lyrics_matrix[i,j])
      # print(lyrics_matrix[i,j])
   array.append(sub_array)  
   # print(sub_array)   

# print(array)

In [511]:
distance_matrix = distance.cdist(array, array, metric='euclidean')
print(distance_matrix.shape)
print(distance_matrix)

(1000, 1000)
[[0.         1.41280312 1.40827838 ... 1.40699558 1.37655945 1.40296841]
 [1.41280312 0.         1.41084217 ... 1.3914612  1.40307641 1.39849128]
 [1.40827838 1.41084217 0.         ... 1.39777947 1.40865373 1.40160343]
 ...
 [1.40699558 1.3914612  1.39777947 ... 0.         1.39778382 1.2963949 ]
 [1.37655945 1.40307641 1.40865373 ... 1.39778382 0.         1.40706058]
 [1.40296841 1.39849128 1.40160343 ... 1.2963949  1.40706058 0.        ]]


In [512]:
similarities_distance = {}
for i in range(len(distance_matrix)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = distance_matrix[i].argsort()[:50] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities_distance[songs['song'].iloc[i]] = [(distance_matrix[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]
# print(similarities_distance)

In [513]:
class ContentBasedRecommender_Euclidean:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended (Euclidean) songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [514]:
recommedations_Euclidean = ContentBasedRecommender_Euclidean(similarities_distance)


In [515]:
recommendation = {
    "song": songs['song'].iloc[1],
    "number_songs": 8
}

recommedations_Euclidean.recommend(recommendation)



The 8 recommended (Euclidean) songs for Andante, Andante are:
Number 1:
Gonna Sing You My Lovesong by ABBA with 1.284 similarity score
--------------------
Number 2:
Cowboy Song by Arlo Guthrie with 1.319 similarity score
--------------------
Number 3:
Making Love Out Of Nothing At All by Air Supply with 1.323 similarity score
--------------------
Number 4:
Dance by ABBA with 1.328 similarity score
--------------------
Number 5:
Don't Let It Get You Down by America with 1.329 similarity score
--------------------
Number 6:
Strong Strong Wind by Air Supply with 1.331 similarity score
--------------------
Number 7:
Power Of Love by Arlo Guthrie with 1.331 similarity score
--------------------
Number 8:
Make It Right by Air Supply with 1.332 similarity score
--------------------


In [519]:
recommendation3 = {
    "song": songs['song'].iloc[800],
    "number_songs": 8 
}

recommedations_Euclidean.recommend(recommendation3)

The 8 recommended (Euclidean) songs for I Ride On Old Pain are:
Number 1:
Same Old Song And Dance by Aerosmith with 1.285 similarity score
--------------------
Number 2:
Old Friend by Allman Brothers Band with 1.301 similarity score
--------------------
Number 3:
Milkcow Blues by Aerosmith with 1.31 similarity score
--------------------
Number 4:
Let Me Ride by Allman Brothers Band with 1.318 similarity score
--------------------
Number 5:
The Motorcycle Song by Arlo Guthrie with 1.321 similarity score
--------------------
Number 6:
When The Cactus Is In Bloom by Arlo Guthrie with 1.324 similarity score
--------------------
Number 7:
Draw The Line by Aerosmith with 1.325 similarity score
--------------------
Number 8:
Down To The Water by America with 1.325 similarity score
--------------------


In [517]:
arr = [[1,0,0], [0.9,0.07,0.03], [0.02, 0.96, 0.04], [0.1, 0.85, 0.05]]
testArray = distance.cdist(arr, arr, metric='euclidean')
# print(testArray.shape)
print(testArray)

similarities_distance = {}
for i in range(len(testArray)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    # Return index asc
    similar_indices = testArray[i].argsort()[:3] 
    # similar_indices = testArray[i].argsort()
    print(similar_indices)

    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities_distance[songs['song'].iloc[i]] = [(testArray[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]
    print('this is:',songs['song'].iloc[i], [(testArray[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][0:])
# print(similarities_distance)

[[0.         0.12569805 1.37244308 1.23895117]
 [0.12569805 0.         1.25163893 1.1174972 ]
 [1.37244308 1.25163893 0.         0.13638182]
 [1.23895117 1.1174972  0.13638182 0.        ]]
[0 1 3]
this is: Ahe's My Kind Of Girl [(0.0, "Ahe's My Kind Of Girl", 'ABBA'), (0.12569805089976535, 'Andante, Andante', 'ABBA'), (1.2389511693363866, 'Bang', 'ABBA')]
[1 0 3]
this is: Andante, Andante [(0.0, 'Andante, Andante', 'ABBA'), (0.12569805089976535, "Ahe's My Kind Of Girl", 'ABBA'), (1.1174972035759194, 'Bang', 'ABBA')]
[2 3 1]
this is: As Good As New [(0.0, 'As Good As New', 'ABBA'), (0.13638181696985854, 'Bang', 'ABBA'), (1.2516389255691913, 'Andante, Andante', 'ABBA')]
[3 2 1]
this is: Bang [(0.0, 'Bang', 'ABBA'), (0.13638181696985854, 'As Good As New', 'ABBA'), (1.1174972035759194, 'Andante, Andante', 'ABBA')]
